In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.tabular import *


In [3]:
import pandas as pd
import numpy as np

# Load data
train = pd.read_csv('./train/train.csv')
test = pd.read_csv('./test/test.csv')

print(train.shape)
print(test.shape)

(300000, 25)
(200000, 24)


In [4]:
train.head()

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,...,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
0,0,0,0,0,T,Y,Green,Triangle,Snake,Finland,...,2f4cb3d51,2,Grandmaster,Cold,h,D,kr,2,2,0
1,1,0,1,0,T,Y,Green,Trapezoid,Hamster,Russia,...,f83c56c21,1,Grandmaster,Hot,a,A,bF,7,8,0
2,2,0,0,0,F,Y,Blue,Trapezoid,Lion,Russia,...,ae6800dd0,1,Expert,Lava Hot,h,R,Jc,7,2,0
3,3,0,1,0,F,Y,Red,Trapezoid,Snake,Canada,...,8270f0d71,1,Grandmaster,Boiling Hot,i,D,kW,2,1,1
4,4,0,0,0,F,N,Red,Trapezoid,Lion,Canada,...,b164b72a7,1,Grandmaster,Freezing,a,R,qP,7,8,0


In [5]:
test.head()

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,...,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
0,300000,0,0,1,T,Y,Blue,Triangle,Axolotl,Finland,...,9d117320c,3c49b42b8,2,Novice,Warm,j,P,be,5,11
1,300001,0,0,0,T,N,Red,Square,Lion,Canada,...,46ae3059c,285771075,1,Master,Lava Hot,l,A,RP,7,5
2,300002,1,0,1,F,Y,Blue,Square,Dog,China,...,b759e21f0,6f323c53f,2,Expert,Freezing,a,G,tP,1,12
3,300003,0,0,1,T,Y,Red,Star,Cat,China,...,0b6ec68ff,b5de3dcc4,1,Contributor,Lava Hot,b,Q,ke,2,3
4,300004,0,1,1,F,N,Red,Trapezoid,Dog,China,...,f91f3b1ee,967cfa9c9,3,Grandmaster,Lava Hot,l,W,qK,4,11


In [6]:
target = train['target']
train_id = train['id']
test_id = test['id']
train.drop('id', axis=1, inplace=True)
test.drop('id', axis=1, inplace=True)

In [30]:
rang = {"Grandmaster" : 4, "Master" : 3, "Expert" : 2, "Contributor" : 1, "Novice" : 0}
temperature = {"Freezing" : 0, "Cold": 1, "Warm" : 2, "Hot": 3, "Boiling Hot" : 4, "Lava Hot" : 5}

In [31]:
train['ord_1'] = train['ord_1'].map(rang)
train['ord_2'] = train['ord_2'].map(temperature)
train['ord_3'] = train['ord_3'].map({val : idx for idx, val in enumerate(np.unique(train['ord_3']))})
train['ord_4'] = train['ord_4'].map({val : idx for idx, val in enumerate(np.unique(train['ord_4']))})
train['ord_5_new_1'] = train['ord_5'].apply(lambda x: x[0])
train['ord_5_new_1'] = train['ord_5_new_1'].map({val : idx for idx, val in enumerate(np.unique(train['ord_5_new_1']))})
train['ord_5_new_2'] = train['ord_5'].apply(lambda x: x[1])
train['ord_5_new_2'] = train['ord_5_new_2'].map({val : idx for idx, val in enumerate(np.unique(train['ord_5_new_2']))})
train['ord_5_new_2'] = train['ord_5_new_2'].map({val : idx for idx, val in enumerate(np.unique(train['ord_5_new_2']))})
train['ord_5'] = train['ord_5_new_2']
train.drop('ord_5_new_2', axis=1, inplace=True)

In [36]:
train.head()

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,...,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target,ord_5_new_1
0,0,0,0,T,Y,Green,Triangle,Snake,Finland,Bassoon,...,2,4,1,7,3,43,2,2,0,36
1,0,1,0,T,Y,Green,Trapezoid,Hamster,Russia,Piano,...,1,4,3,0,0,5,7,8,0,27
2,0,0,0,F,Y,Blue,Trapezoid,Lion,Russia,Theremin,...,1,2,5,7,17,28,7,2,0,9
3,0,1,0,F,Y,Red,Trapezoid,Snake,Canada,Oboe,...,1,4,4,8,3,22,2,1,1,36
4,0,0,0,F,N,Red,Trapezoid,Lion,Canada,Oboe,...,1,4,0,0,17,15,7,8,0,42


In [33]:
cat_vars = ['nom_0', 'nom_1', 'nom_2', 'nom_3', 'nom_4', 'nom_5', 'nom_6', 'nom_7', 'nom_8', 'nom_9', 'bin_0', 'bin_1', 'bin_2', 'bin_3', 'bin_4']
cont_vars = ['ord_0', 'ord_1', 'ord_2', 'ord_3', 'ord_4', 'ord_5', 'day', 'month']

In [34]:
dep_var = 'target'
procs = [FillMissing, Categorify, Normalize]

In [35]:
path = Config.data_path()/'categorical'

In [37]:
data = (TabularList.from_df(train, path=path, cat_names=cat_vars, cont_names=cont_vars, procs=procs)
                           .split_by_rand_pct(0.2)
                           .label_from_df(cols=dep_var)
                           .databunch())

In [38]:
data.show_batch(rows=10)

nom_0,nom_1,nom_2,nom_3,nom_4,nom_5,nom_6,nom_7,nom_8,nom_9,bin_0,bin_1,bin_2,bin_3,bin_4,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
Green,Star,Axolotl,Finland,Theremin,dc07effb0,c3517eaeb,f2935290e,a142509f1,e95678273,0,1,1,T,Y,2.1357,-0.9675,0.8159,1.9310,1.2581,0.2103,-0.0050,0.8390,0
Green,Square,Snake,Russia,Bassoon,bdb9fef4a,0b23e3352,7b22090e2,06e189096,b556940dc,0,1,0,T,N,-0.6728,-0.9675,-1.1524,0.3638,-0.5180,-0.1465,-0.5546,1.6193,0
Red,Circle,Cat,Canada,Piano,b7bb45938,bc31d6a6d,99a4414f3,09b25b0ac,724214461,0,0,1,T,Y,-0.6728,1.4173,-1.1524,1.1474,-1.6110,-0.5747,1.0943,-1.2417,0
Green,Trapezoid,Lion,Finland,Piano,c0534106d,2a22b315e,a354255c4,87f68ba07,f1f881e1c,1,0,0,F,N,-0.6728,1.4173,-1.1524,-0.1586,-1.4744,-0.5747,-0.5546,1.6193,0
Red,Trapezoid,Cat,Canada,Theremin,de04b8750,b238965eb,a9fc363fb,26a23df3e,a232004fa,0,1,1,T,N,0.7315,1.4173,-0.1682,-0.1586,-1.6110,0.5671,-0.5546,1.6193,0
Blue,Square,Lion,Canada,Oboe,493de6347,5fd7600de,2c89fa25b,07a517154,0d6d240e7,0,1,0,F,N,-0.6728,0.2249,1.3080,0.8862,-0.3814,-1.5025,-0.5546,1.3592,0
Red,Trapezoid,Cat,Russia,Theremin,f50eb07ae,ec5064acb,cda68a782,7bd5180a7,92310933e,0,0,1,T,N,-0.6728,0.2249,0.8159,-1.7257,1.5314,1.7804,2.1937,-0.9816,0
Blue,Trapezoid,Cat,China,Piano,dbfb714a4,882ce2ad7,d998d31f1,7431726cf,8dd7b712c,0,1,0,F,N,2.1357,0.2249,-0.1682,0.6250,-1.7476,1.4949,-0.5546,1.6193,1
Blue,Trapezoid,Cat,Russia,Oboe,f0c1ceee4,b48077cc4,ec54fcb5c,b35f308e6,cafbf9566,0,0,1,T,N,-0.6728,0.8211,1.3080,0.8862,0.8483,-0.4320,0.5447,-0.7215,0
Green,Polygon,Snake,India,Bassoon,be4578201,d76a148ab,7df20ed82,ad97c3d39,01d331fac,0,0,0,F,N,-0.6728,-0.3713,0.3238,0.3638,0.9849,1.7090,-0.0050,-0.4614,0


In [41]:
learn = tabular_learner(data, layers=[10], metrics=[accuracy, AUROC()])#, ps=[0.1,0.5], emb_drop=0.04)

In [42]:
learn.fit(10, 1e-3)

epoch,train_loss,valid_loss,accuracy,auroc,time
0,0.518404,0.508563,0.747383,0.770612,02:45
1,0.503945,0.511293,0.746783,0.768143,02:46
2,0.480685,0.521680,0.740750,0.760091,02:45
3,0.466468,0.534740,0.731917,0.749678,02:45
4,0.446827,0.552445,0.727183,0.742383,02:45
5,0.425004,0.567120,0.721383,0.736243,02:45
6,0.422414,0.589389,0.714667,0.722961,02:43
7,0.398957,0.607906,0.704883,0.720811,02:38
8,0.381144,0.629783,0.704583,0.713351,02:45
9,0.379182,0.647336,0.702200,0.704683,02:45


In [ ]:
learn.recorder.plot_losses()